In [229]:
from datetime import datetime, timedelta
# A. ADD THESE CRITICAL IMPORTS
from mailwizz.endpoint.campaigns import Campaigns
from mailwizz.endpoint.lists import Lists
from mailwizz.endpoint.list_subscribers import ListSubscribers
from mailwizz.config import Config # Import the Config class
from mailwizz.base import Base     # Import the Base class
from mailwizz.endpoint.lists import Lists # (Kept this, but it's not strictly necessary for Campaigns)
import json
import pandas as pd 

def setup():
    # configuration object
    config = Config({
        'api_url': 'https://mcharge.karmagroup.com/api/index.php',
        'public_key': '180c97ec2bb6fe02ef5047d300591092676e8bb8',
        'private_key': '180c97ec2bb6fe02ef5047d300591092676e8bb8',
        'charset': 'utf-8'
    })

    # now inject the configuration and we are ready to make api calls
    Base.set_config(config)

# B. EXECUTE THE SETUP FUNCTION TO APPLY THE CONFIGURATION
setup()

In [230]:
"""
CREATE THE ENDPOINT
"""
endpoint_list = Lists()

"""
GET ALL ITEMS
"""
response_list = endpoint_list.get_lists(page=1, per_page=10)

"""
DISPLAY RESPONSE
"""
print(response_list.content)
data_list = response_list.content

b'{"status":"success","data":{"count":"8","total_pages":1,"current_page":1,"next_page":null,"prev_page":null,"records":[{"general":{"list_uid":"oc390b3bhb422","name":"KCEX | Lawrence Blair at Karma Kandara","display_name":"Curated Events Auto","description":"Curated Events Auto"},"defaults":{"from_email":"adhitya.putra@karmagroup.com","from_name":"Adhitya Putra Utomo","reply_to":"adhitya.putra@karmagroup.com","subject":""},"notifications":{"subscribe":"no","unsubscribe":"no","subscribe_to":"","unsubscribe_to":""},"company":{"name":"Karma Resorts","address_1":"Jalan By Pass Ngurah Rai","address_2":"","zone_name":"","city":"Bali","zip_code":"80361","phone":"","address_format":"[COMPANY_NAME]\\n[COMPANY_ADDRESS_1] [COMPANY_ADDRESS_2]\\n[COMPANY_CITY] [COMPANY_ZONE] [COMPANY_ZIP]\\n[COMPANY_COUNTRY]\\n[COMPANY_WEBSITE]","country":{"country_id":"100","name":"Indonesia","code":"ID"}}},{"general":{"list_uid":"ev168y7nhl0eb","name":"KCEX | Lawrence Blair at Karma Kandara","display_name":"Curat

In [231]:
def extract_list_data_to_dataframe(data_list: str) -> pd.DataFrame:
    """
    Parses the Mailwizz lists JSON response and extracts list UIDs and names
    into a pandas DataFrame.

    Args:
        json_string: The raw JSON string received from the API.

    Returns:
        A pandas DataFrame with 'list_uid' and 'name' columns, or an empty
        DataFrame on failure.
    """
    # Define an empty DataFrame structure for error cases
    empty_df = pd.DataFrame(columns=['list_uid', 'name'])

    try:
        # 1. Load the JSON string into a Python dictionary
        data = json.loads(data_list)

        # 2. Safely access the nested 'records' list
        records = data.get('data', {}).get('records', [])

        if not records:
            print("No list records found in the data.")
            return empty_df

        # 3. Extract the required data points into a list of dictionaries
        list_data = []
        for record in records:
            general = record.get('general', {})
            list_data.append({
                'list_uid': general.get('list_uid'),
                'name': general.get('name')
            })

        # 4. Convert the list of dictionaries into a pandas DataFrame
        df = pd.DataFrame(list_data)

        # 5. Clean up: Filter out any rows where the name or UID might be missing
        df.dropna(subset=['list_uid', 'name'], inplace=True)

        return df

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return empty_df
    except Exception as e:
        print(f"An unexpected error occurred during DataFrame creation: {e}")
        return empty_df

if __name__ == "__main__":

    # Call the extraction function
    list_dataframe = extract_list_data_to_dataframe(data_list)

    if not list_dataframe.empty:
        print("\n--- Extracted List Data (Pandas DataFrame) ---")
        print(f"Total lists extracted: {len(list_dataframe)}")
        print("\nDataFrame Head:")
        display(list_dataframe)
        
        # Example: Save to a CSV file
        # list_dataframe.to_csv('mailwizz_mailing_lists.csv', index=False)
        # print("\nData also saved to mailwizz_mailing_lists.csv")
    else:
        print("Could not create the DataFrame.")


--- Extracted List Data (Pandas DataFrame) ---
Total lists extracted: 8

DataFrame Head:


,list_uid,name
0,oc390b3bhb422,KCEX | Lawrence Blair at Karma Kandara
1,ev168y7nhl0eb,KCEX | Lawrence Blair at Karma Kandara
2,vv530s9gyrfae,"KCEX | Sip, Dine and Create: Bangalore"
3,pl253v6t83a39,KCEX | Candlelight Concert's Best of Fleetwood...
4,vp58496ejw16e,KCEX | Titans of Rugby at Twickenham
5,ns486t973bffb,"KCEX | Sip, Dine and Create: Hyderabad"
6,pr592vvrzb7c2,Curated Events Auto (TEST)
7,cy330jczo8023,Curated Events Auto


In [243]:
from mailwizz.endpoint.list_segments import ListSegments

"""
CREATE THE ENDPOINT
"""
endpoint_segments = ListSegments()

"""
GET ALL SEGMENTS OF A LIST
"""
response_segments = endpoint_segments.get_segments(list_uid='vv530s9gyrfae')

"""
DISPLAY RESPONSE
"""
print(response_segments.content)
data_segment = response_segments.content

b'{"status":"success","data":{"count":"4","total_pages":1,"current_page":1,"next_page":null,"prev_page":null,"records":[{"segment_uid":"pm702n00pccc1","name":"Generic Bali","subscribers_count":2},{"segment_uid":"yc8705vxps3f4","name":"Generic India","subscribers_count":8},{"segment_uid":"gm703x71tac31","name":"KRR Bali","subscribers_count":0},{"segment_uid":"gd559887ze8ae","name":"KRR India","subscribers_count":7}]}}'


In [244]:
def extract_segment_data_to_dataframe(data_segment: bytes) -> pd.DataFrame:
    """
    Parses the Mailwizz segment JSON response, extracts all fields,
    and converts them into a pandas DataFrame.

    Args:
        json_bytes: The raw JSON byte string received from the API (e.g., b'{...}').

    Returns:
        A pandas DataFrame containing all segment fields, or an empty
        DataFrame on failure.
    """
    # Define an empty DataFrame structure for error cases (using sample columns)
    empty_df = pd.DataFrame(columns=['segment_uid', 'name', 'subscribers_count'])

    try:
        # 1. Decode the byte string to a regular JSON string
        json_string = data_segment.decode('utf-8')

        # 2. Load the JSON string into a Python dictionary
        data = json.loads(json_string)

        # 3. Safely access the nested 'records' list
        # This list contains dictionaries, each representing a single segment record
        records = data.get('data', {}).get('records', [])

        if not records:
            print("No segment records found in the data.")
            return empty_df

        # 4. Convert the list of dictionaries (records) directly into a pandas DataFrame
        # This works perfectly because all keys are at the top level of each record.
        df = pd.DataFrame(records)

        # 5. Convert 'subscribers_count' to a numeric type for easier analysis
        if 'subscribers_count' in df.columns:
            df['subscribers_count'] = pd.to_numeric(df['subscribers_count'], errors='coerce')

        # 6. Set the segment_uid as the index
        if 'segment_uid' in df.columns:
            df.set_index('segment_uid', inplace=True)

        return df

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return empty_df
    except Exception as e:
        print(f"An unexpected error occurred during DataFrame creation: {e}")
        return empty_df

if __name__ == "__main__":

    # Call the extraction function
    segment_dataframe = extract_segment_data_to_dataframe(data_segment)

    if not segment_dataframe.empty:
        print("\n--- Extracted Segment Data (Pandas DataFrame) ---")
        print(f"Total segments extracted: {len(segment_dataframe)}")
        print(f"Total subscribers across all segments: {segment_dataframe['subscribers_count'].sum()}")
        print("\nDataFrame:")
        # Display the full DataFrame
        pd.set_option('display.max_columns', None)
        display(segment_dataframe)
    else:
        print("Could not create the DataFrame.")


--- Extracted Segment Data (Pandas DataFrame) ---
Total segments extracted: 4
Total subscribers across all segments: 17

DataFrame:


,name,subscribers_count
segment_uid,,
pm702n00pccc1,Generic Bali,2
yc8705vxps3f4,Generic India,8
gm703x71tac31,KRR Bali,0
gd559887ze8ae,KRR India,7


In [245]:
"""
GET ALL SUBSCRIBERS OF A LIST SEGMENT
"""
response_subs_detail = endpoint_segments.get_subscribers(list_uid='vv530s9gyrfae', segment_uid='yc8705vxps3f4', page=1, per_page=10)

"""
DISPLAY RESPONSE
"""
print(response_subs_detail.content)
data_subs_detail = response_subs_detail.content

b'{"status":"success","data":{"count":8,"total_pages":1,"current_page":1,"next_page":null,"prev_page":null,"records":[{"subscriber_uid":"qa116f4n169f5","FULLNAME_LINK":"Sajeev%20Anand%20Palanivel","SERVICE_OFFICE":"KCI","MEMBER_TYPE":"Karma Club","MEMBER_NO":"1081962","EVENT_DATE_FROM":"2025-11-16","EVENT_DATE_TO":"2025-11-16","EMAIL":"ramyarajentharan@gmail.com","SERVICE_REGION":"INDIA","EVENT_NAME_LINK":"Sip%2C%20Dine%20and%20Create%3A%20Bangalore%20%2816%20Nov%202025%29","EVENT_TARGET":"GENERIC_INDIA","EVENT_DATES_DETAILS":"16 Nov 2025","EVENT_NAME_DETAILS":"Sip, Dine and Create: Bangalore","EVENT_IMG_URL":"https:\\/\\/edm.karmacommunity.com\\/karmagroup\\/2025\\/Karma-Curated-Experiences\\/post_event_header_images\\/header_Shuck_&_Sip_Sydney_Oyster_Experience_(22_Feb_2025).jpg","IS_MEMBER":"Yes","EVENT_NAME":"SIP, DINE AND CREATE: BANGALORE 16 NOV","FNAME":"Sajeev Anand","LNAME":"Palanivel","status":"confirmed","source":"api","ip_address":"","date_added":"2025-11-17 05:32:49"},{"su

In [246]:
import sys
import urllib.parse

def extract_subscriber_data_to_dataframe(data_subs_detail: bytes) -> pd.DataFrame:
    """
    Parses the Mailwizz subscriber JSON response, extracts all fields,
    and converts them into a pandas DataFrame.

    It specifically handles URL-decoding for fields like FULLNAME_LINK and 
    sets the 'subscriber_uid' as the index.

    Args:
        json_bytes: The raw JSON byte string received from the API (e.g., b'{...}').

    Returns:
        A pandas DataFrame containing all subscriber fields, or an empty
        DataFrame on failure.
    """
    # Define an empty DataFrame structure for error cases (using sample columns)
    empty_df = pd.DataFrame(columns=['subscriber_uid', 'EMAIL', 'FNAME', 'EVENT_NAME'])

    try:
        # 1. Decode the byte string to a regular JSON string
        json_string = data_subs_detail.decode('utf-8')

        # 2. Load the JSON string into a Python dictionary
        data = json.loads(json_string)

        # 3. Safely access the nested 'records' list
        records = data.get('data', {}).get('records', [])

        if not records:
            print("No subscriber records found in the data.")
            return empty_df

        # 4. Convert the list of dictionaries (records) directly into a pandas DataFrame
        df = pd.DataFrame(records)

        # 5. Clean up: URL-decode fields that look URL-encoded.
        # We'll explicitly check for a few common ones.
        for col in ['FULLNAME_LINK', 'EVENT_NAME_LINK']:
            if col in df.columns:
                # Use urllib.parse.unquote to decode %-encoded strings
                df[col] = df[col].apply(urllib.parse.unquote)

        # 6. Set the subscriber_uid as the index
        if 'subscriber_uid' in df.columns:
            df.set_index('subscriber_uid', inplace=True)

        return df

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return empty_df
    except Exception as e:
        print(f"An unexpected error occurred during DataFrame creation: {e}")
        return empty_df

if __name__ == "__main__":

    # Call the extraction function
    subscriber_dataframe = extract_subscriber_data_to_dataframe(data_subs_detail)

    if not subscriber_dataframe.empty:
        display(subscriber_dataframe)
    else:
        print("Could not create the DataFrame.")

,FULLNAME_LINK,SERVICE_OFFICE,MEMBER_TYPE,MEMBER_NO,EVENT_DATE_FROM,EVENT_DATE_TO,EMAIL,SERVICE_REGION,EVENT_NAME_LINK,EVENT_TARGET,EVENT_DATES_DETAILS,EVENT_NAME_DETAILS,EVENT_IMG_URL,IS_MEMBER,EVENT_NAME,FNAME,LNAME,status,source,ip_address,date_added
subscriber_uid,,,,,,,,,,,,,,,,,,,,,
qa116f4n169f5,Sajeev Anand Palanivel,KCI,Karma Club,1081962,2025-11-16,2025-11-16,ramyarajentharan@gmail.com,INDIA,"Sip, Dine and Create: Bangalore (16 Nov 2025)",GENERIC_INDIA,16 Nov 2025,"Sip, Dine and Create: Bangalore",https://edm.karmacommunity.com/karmagroup/2025...,Yes,"SIP, DINE AND CREATE: BANGALORE 16 NOV",Sajeev Anand,Palanivel,confirmed,api,,2025-11-17 05:32:49
yb0354j0ja647,Gaurav Jain,KCI,Karma Club,131808,2025-11-16,2025-11-16,archiorchid@gmail.com,INDIA,"Sip, Dine and Create: Bangalore (16 Nov 2025)",GENERIC_INDIA,16 Nov 2025,"Sip, Dine and Create: Bangalore",https://edm.karmacommunity.com/karmagroup/2025...,Yes,"SIP, DINE AND CREATE: BANGALORE 16 NOV",Gaurav,Jain,confirmed,api,74.125.209.66,2025-11-17 05:32:49
tg9175mroq39a,Thippeswamy S,KCI,Karma Club,1207570,2025-11-16,2025-11-16,s.thippeswamy@gmail.com,INDIA,"Sip, Dine and Create: Bangalore (16 Nov 2025)",GENERIC_INDIA,16 Nov 2025,"Sip, Dine and Create: Bangalore",https://edm.karmacommunity.com/karmagroup/2025...,Yes,"SIP, DINE AND CREATE: BANGALORE 16 NOV",Thippeswamy,S,confirmed,api,172.225.186.9,2025-11-17 05:32:49
kc7639lmzy903,Latha K,KCI,Karma Club,1140633,2025-11-16,2025-11-16,laki.lata@gmail.com,INDIA,"Sip, Dine and Create: Bangalore (16 Nov 2025)",GENERIC_INDIA,16 Nov 2025,"Sip, Dine and Create: Bangalore",https://edm.karmacommunity.com/karmagroup/2025...,Yes,"SIP, DINE AND CREATE: BANGALORE 16 NOV",Latha,K,confirmed,api,74.125.209.66,2025-11-17 05:32:49
ss410trsy099c,Sunil Panjwani,KCI,Karma Club,56007,2025-11-16,2025-11-16,sunillynette.panjwani@gmail.com,INDIA,"Sip, Dine and Create: Bangalore (16 Nov 2025)",GENERIC_INDIA,16 Nov 2025,"Sip, Dine and Create: Bangalore",https://edm.karmacommunity.com/karmagroup/2025...,Yes,"SIP, DINE AND CREATE: BANGALORE 16 NOV",Sunil,Panjwani,confirmed,api,101.0.63.244,2025-11-17 05:32:50
st749o8ok363e,"C, Vinay & M, Pavitra",KCI,Karma Club,1134586,2025-11-16,2025-11-16,vinay.chikkaiah@gmail.com,INDIA,"Sip, Dine and Create: Bangalore (16 Nov 2025)",GENERIC_INDIA,16 Nov 2025,"Sip, Dine and Create: Bangalore",https://edm.karmacommunity.com/karmagroup/2025...,Yes,"SIP, DINE AND CREATE: BANGALORE 16 NOV","C, Vinay & M, Pavitra",,confirmed,api,74.125.209.65,2025-11-17 05:32:50
zx241j2jpb30d,Pramod Narasimha Murthy,KCI,Karma Club,1070470,2025-11-16,2025-11-16,pommie12@gmail.com,INDIA,"Sip, Dine and Create: Bangalore (16 Nov 2025)",GENERIC_INDIA,16 Nov 2025,"Sip, Dine and Create: Bangalore",https://edm.karmacommunity.com/karmagroup/2025...,Yes,"SIP, DINE AND CREATE: BANGALORE 16 NOV",Pramod Narasimha,Murthy,confirmed,api,,2025-11-17 05:32:50
ht868g3crv8b8,Fajar Fatoni,KCI,Karma Club,1081962,2025-11-16,2025-11-16,fajar.fatoni@karmagroup.com,INDIA,"Sip, Dine and Create: Bangalore (16 Nov 2025)",GENERIC_INDIA,16 Nov 2025,"Sip, Dine and Create: Bangalore",https://edm.karmacommunity.com/karmagroup/2025...,Yes,"SIP, DINE AND CREATE: BANGALORE 16 NOV",Fajar,Fatoni,confirmed,api,74.125.209.67,2025-11-17 05:32:50


In [247]:
"""
CREATE THE endpoint_campaigns
"""
endpoint_campaigns = Campaigns()

"""
GET ALL ITEMS
"""
response_campaigns = endpoint_campaigns.get_campaigns(page=1, per_page=10)

"""
DISPLAY RESPONSE
"""
print(response_campaigns.content)
data_campaign = response_campaigns.content

b'{"status":"success","data":{"count":"14","total_pages":2,"current_page":1,"next_page":2,"prev_page":null,"records":[{"campaign_uid":"qw532w7w3845e","campaign_id":"3508","type":"regular","name":"KCEX | Lawrence Blair at Karma Kandara - KRR_BALI - 17 Nov 2025 - 19 Nov 2025","status":"sent","group":[]},{"campaign_uid":"po179qjf5x94e","campaign_id":"3502","type":"regular","name":"KCEX | Lawrence Blair at Karma Kandara - KRR_BALI - 17 Nov 2025 - 19 Nov 2025","status":"sent","group":[]},{"campaign_uid":"gz709y5zvkd5a","campaign_id":"3501","type":"regular","name":"KCEX | Sip, Dine and Create: Bangalore - KRR_INDIA - 16 Nov 2025","status":"sent","group":[]},{"campaign_uid":"hz490158jt9b1","campaign_id":"3500","type":"regular","name":"KCEX | Sip, Dine and Create: Bangalore - GENERIC_INDIA - 16 Nov 2025","status":"sent","group":[]},{"campaign_uid":"st563o6acs150","campaign_id":"3499","type":"regular","name":"KCEX | Sip, Dine and Create: Bangalore - GENERIC_BALI - 16 Nov 2025","status":"sent","

In [248]:
# This function simulates the API response for testing purposes, 
# but will be replaced by the actual 'response.content' in a live environment.
def get_campaign_records(content_bytes):
    # In a real environment, you would use 'response.content.decode("utf-8")'
    return content_bytes.decode("utf-8")

# --- MAIN LOGIC ---

In [249]:
# --- MAIN LOGIC ---

# 1. Get and parse the JSON string
json_str = get_campaign_records(data_campaign)

try:
    data = json.loads(json_str)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    exit()

# 2. Extract the list of records
campaign_records = data['data']['records']

# 3. Create the Pandas DataFrame (unnormalized - still needed for normalization step)
df = pd.DataFrame(campaign_records)

# 4. Normalize the 'group' column to expand details into separate columns
df_normalized = pd.json_normalize(campaign_records, sep='_')

# 5. Clean up column names and drop the original nested column
if 'group_group_uid' in df_normalized.columns:
    df_normalized = df_normalized.rename(columns={
        'group_group_uid': 'group_uid',
        'group_name': 'group_name'
    })
    
df_normalized = df_normalized.drop(columns=['group'], errors='ignore')

# 6. Display the resulting DataFrame in Jupyter:
# We only display the final, normalized DataFrame as requested.

print("Final Normalized Campaign DataFrame (df_normalized):")
df_normalized

Final Normalized Campaign DataFrame (df_normalized):


,campaign_uid,campaign_id,type,name,status,group_uid,group_name
0,qw532w7w3845e,3508,regular,KCEX | Lawrence Blair at Karma Kandara - KRR_B...,sent,NaN,NaN
1,po179qjf5x94e,3502,regular,KCEX | Lawrence Blair at Karma Kandara - KRR_B...,sent,NaN,NaN
2,gz709y5zvkd5a,3501,regular,"KCEX | Sip, Dine and Create: Bangalore - KRR_I...",sent,NaN,NaN
3,hz490158jt9b1,3500,regular,"KCEX | Sip, Dine and Create: Bangalore - GENER...",sent,NaN,NaN
4,st563o6acs150,3499,regular,"KCEX | Sip, Dine and Create: Bangalore - GENER...",sent,NaN,NaN
5,ne343p9mhrc04,3498,regular,KCEX | Candlelight Concert's Best of Fleetwood...,sent,NaN,NaN
6,xg771ea0ft673,3497,regular,KCEX | Candlelight Concert's Best of Fleetwood...,sent,NaN,NaN
7,ew083mjsdw036,3494,regular,Titans of Rugby at Twickenham - 1 KRR_BALIN...,sent,NaN,NaN
8,we312kebmwcad,3493,regular,"Sip, Dine and Create: Hyderabad - 0 KRR_IND...",sent,NaN,NaN
9,ph837zprorfcf,3492,regular,TEST,sent,qh200c4x3eb54,Karma Resorts


In [257]:
from mailwizz.endpoint.campaigns_tracking import CampaignsTracking

"""
CREATE THE ENDPOINT
"""
endpoint_campaigns_tracking = CampaignsTracking()

"""
Track subscriber click for campaign click
"""
response_campaigns_tracking = endpoint_campaigns_tracking.track_opening(campaign_uid='hz490158jt9b1', subscriber_uid='qa116f4n169f5')

"""
DISPLAY RESPONSE
"""
print(response_campaigns_tracking.content)

b'{"status":"success"}'
